# NY Senate Analysis

### This notebook is divided into several sections. The first takes the most recent CSV files produced by the Senate Scraper and concatenates them into one cumulative dataframe and csv file. 

### The second analyzes for the last legislative action (lla) temporal differences between session.

### The third analyzes veto data - how many vetoes and if the senate vote count could override a veto

In [1]:
import pandas as pd

In [2]:
df_2011 = pd.read_csv("2011_legis_data_4.12.2021.csv", sep="|")
df_2013 = pd.read_csv("2013_legis_data_4.12.2021.csv", sep="|")
df_2015 = pd.read_csv("2015_legis_data_4.12.2021.csv", sep="|")
df_2017 = pd.read_csv("2017_legis_data_4.12.2021.csv", sep="|")
df_2019 = pd.read_csv("2019_legis_data_4.12.2021.csv", sep="|")

In [3]:
frames = [df_2011,df_2013,df_2015,df_2017,df_2019]
df = pd.concat(frames)

### This section spits out totals for signed and vetoed values for each 2-year session.
#### I did this to compare my numbers with the NYPIRG. I had a scare when I compared a different section of my data to theirs -- it was way off. But these are much closer, usually within ten bills or so. So my working theory is for that other data set, despite what seemed to be the same thing, was actually different.

In [4]:
df.shape

(42300, 20)

In [5]:
### 2011-2012 legislative session signed_by_gov "true" value counts ### NYPIRG 1,115
df_2011[df_2011.signed_by_gov == True].shape

(1111, 20)

In [6]:
### 2011-2012 legislative session vetoed_by_gov "true" value counts ### NYPIRG 129
df_2011[df_2011.vetoed_by_gov == True].shape

(126, 20)

In [7]:
### 2013-2014 legislative session signed_by_gov "true" value counts ### NYPIRG 1,110
df_2013[df_2013.signed_by_gov == True].shape

(1103, 20)

In [8]:
### 2013-2014 legislative session vetoed_by_gov "true" value counts ### NYPIRG 196
df_2013[df_2013.vetoed_by_gov == True].shape

(184, 20)

In [9]:
### 2015-2016 legislative session signed_by_gov "true" value counts ### NYPIRG 1,108
df_2015[df_2015.signed_by_gov == True].shape

(1096, 20)

In [10]:
### 2015-2016 legislative session vetoed_by_gov "true" value counts ### NYPIRG 232
df_2015[df_2015.vetoed_by_gov == True].shape

(217, 20)

In [11]:
### 2017-2018 legislative session signed_by_gov "true" value counts ### NYPIRG 1,027
df_2017[df_2017.signed_by_gov == True].shape

(1008, 20)

In [12]:
### 2017-2018 legislative session vetoed_by_gov "true" value counts ### NYPIRG 223
df_2017[df_2017.vetoed_by_gov == True].shape

(215, 20)

In [13]:
df.shape

(42300, 20)

In [14]:
df.columns

Index(['bill_number', 'session', 'url', 'bill_title', 'spon_sen_name',
       'dist_name', 'aye_vote', 'nay_vote', 'dtg', 'dtg_date', 'lla_needed?',
       'lla_date', 'lla_action', 'signed_by_gov', 'signed_date',
       'vetoed_by_gov', 'vetoed_date', 'sponser_memo', 'sponser_memo2',
       'bill_text'],
      dtype='object')

In [15]:
df.lla_action.value_counts()

FALSE                                                          35892
returned to senate                                              3375
returned to assembly                                            2943
delivered to assembly                                             15
<class 'Exception'>                                                8
referred to ways and means                                         5
referred to codes                                                  4
referred to rules                                                  4
amend (t) and recommit to finance                                  3
amend and recommit to finance                                      2
delivered to senate                                                2
substituted by a775                                                1
substituted by a3675b                                              1
substituted by a3009c                                              1
print number 6353e                

In [16]:
df.to_csv("all_legis_data_4.12.2021.csv", sep="|")

## Last Legislative Action (lla) temporal analysis

Set up a dataframe with only bills that have been delivered to the governor

In [17]:
df.dtg.value_counts()

False    35892
True      6408
Name: dtg, dtype: int64

In [18]:
df_dtg = df[df.dtg == True]

In [19]:
df_dtg.dtg.value_counts()

True    6408
Name: dtg, dtype: int64

In [20]:
df_dtg.columns

Index(['bill_number', 'session', 'url', 'bill_title', 'spon_sen_name',
       'dist_name', 'aye_vote', 'nay_vote', 'dtg', 'dtg_date', 'lla_needed?',
       'lla_date', 'lla_action', 'signed_by_gov', 'signed_date',
       'vetoed_by_gov', 'vetoed_date', 'sponser_memo', 'sponser_memo2',
       'bill_text'],
      dtype='object')

Remove the one error string

In [21]:
df_dtg = df_dtg[~df_dtg.lla_date.str.contains('<class \'Exception\'>')]

Convert to datetime

In [22]:
df_dtg['lla_date'] = pd.to_datetime(df_dtg['lla_date'])
df_dtg['dtg_date'] = pd.to_datetime(df_dtg['dtg_date'])

Create the 'time_to_dtg' column

In [23]:
df_dtg['time_to_dtg'] = df_dtg['dtg_date'] - df_dtg['lla_date']

In [24]:
df_dtg.time_to_dtg.value_counts()

29 days     128
46 days     116
169 days    110
1 days       93
181 days     92
           ... 
229 days      1
253 days      1
273 days      1
252 days      1
265 days      1
Name: time_to_dtg, Length: 244, dtype: int64

# Separate them by session year

### 2011, 2013, 2015, 2017, 2019

Each session runs two years in length

In [25]:
df_dtg.session.value_counts()

2019    1343
2015    1312
2013    1289
2011    1234
2017    1222
Name: session, dtype: int64

## 2011 Data
#### Average time to dtg: 45 days

In [26]:
df_dtg[df_dtg.session == 2011].time_to_dtg.mean()

Timedelta('45 days 12:29:10.405186385')

## 2013 Data
#### Average time to dtg: 81 days

In [27]:
df_dtg[df_dtg.session == 2013].time_to_dtg.mean()

Timedelta('81 days 23:22:01.024049651')

## 2015 Data
#### Average time to dtg: 93 days

In [28]:
df_dtg[df_dtg.session == 2015].time_to_dtg.mean()

Timedelta('93 days 04:23:24.878048780')

## 2017 Data
#### Average time to dtg: 101 days

In [29]:
df_dtg[df_dtg.session == 2017].time_to_dtg.mean()

Timedelta('101 days 11:22:17.479541735')

## 2019 Data
#### Average time to dtg: 114 days

In [30]:
df_dtg[df_dtg.session == 2019].time_to_dtg.mean()

Timedelta('114 days 10:32:36.813104988')

## Breakdown so far:

###### 2011: 45 days
###### 2013: 81 days
###### 2015: 93 days
###### 2017: 101 days
###### 2019: 114 days

In [31]:
df_dtg.shape

(6400, 21)

In [32]:
df_dtg.time_to_dtg.mean()

Timedelta('87 days 18:41:37.500000')

## Other stats, such as min and max value

In [33]:
df_dtg.time_to_dtg.sort_values(ascending=False)

5101   303 days
2494   293 days
6923   286 days
1809   285 days
6651   282 days
         ...   
6880     0 days
7077     0 days
6238     0 days
1983     0 days
6236     0 days
Name: time_to_dtg, Length: 6400, dtype: timedelta64[ns]

In [34]:
df_dtg.loc[[6923]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
6923,senate Bill S6944,2011,https://www.nysenate.gov/legislation/bills/201...,Relates to the definition of certain covered p...,Martin J. Golden,"(R, C, IP) 0 Senate District",60,0,True,2012-06-28,...,2012-06-20,returned to assembly,True,"Jun 29, 2012",False,no date,BILL NUMBER:S6944\n\nTITLE OF BILL:\nAn act\nt...,provides that a hotel or convention center whi...,S6944 (ACTIVE) - Bill Text\n \ndownlo...,8 days
6923,senate Bill S6940,2017,https://www.nysenate.gov/legislation/bills/201...,Prohibits pharmacy benefit managers from restr...,Kemp Hannon,"(R, C, IP) 0 Senate District",60,0,True,2018-12-17,...,2018-03-06,returned to assembly,False,no date,True,"Dec 28, 2018",BILL NUMBER: S6940\n\nSPONSOR: HANNON\n \nTITL...,"Further, this section prohibits the imposition...",S6940 (ACTIVE) - Bill Text\n \ndownlo...,286 days


### Longest in data set

In [35]:
df_dtg.loc[5101]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
5101,senate Bill S5121,2013,https://www.nysenate.gov/legislation/bills/201...,Relates to the demutualization of domestic mut...,Joseph A. Griffo,"(R, C, IP) 47th Senate District",62,0,True,2013-07-19,...,2013-06-17,returned to senate,True,"Jul 31, 2013",False,no date,BILL NUMBER:S5121\n\nTITLE OF BILL: An act to...,report of the company's fair market value by q...,S5121 (ACTIVE) - Bill Text\n \ndownlo...,32 days
5101,senate Bill S5112,2015,https://www.nysenate.gov/legislation/bills/201...,Relates to blackfish or tautog,Kenneth P. LaValle,(R) 0 Senate District,61,0,True,2016-07-13,...,2016-06-08,returned to senate,True,"Jul 21, 2016",False,no date,BILL NUMBER:S5112\n\nTITLE OF BILL: An act to...,"particular, there is no fishing income eligibi...",S5112 (ACTIVE) - Bill Text\n \ndownlo...,35 days
5101,senate Bill S5118B,2017,https://www.nysenate.gov/legislation/bills/201...,Relates to the appointment and promotion of su...,Martin J. Golden,"(R, C, IP) 0 Senate District",60,0,True,2018-12-12,...,2018-02-12,returned to senate,True,"Dec 21, 2018",False,no date,BILL NUMBER: S5118\n\n TITLE OF BILL : An ac...,immediately.\n\n JUSTIFICATION :\n\nThe New Yo...,S5118 - Bill Text\n \ndownload pdf\n\...,303 days
5101,senate Bill S5112A,2019,https://www.nysenate.gov/legislation/bills/201...,Designates a portion of the state highway syst...,Kenneth P. LaValle,(R) 0 Senate District,62,0,True,2019-08-05,...,2019-06-18,returned to senate,True,"Aug 09, 2019",False,no date,BILL NUMBER: S5112\n\nSPONSOR: LAVALLE\n \nTIT...,and maintenance of adequate signing of the sta...,S5112 - Bill Text\n \ndownload pdf\n\...,48 days


In [36]:
df_dtg.time_to_dtg.sort_values(ascending=True)

6236     0 days
1983     0 days
6238     0 days
7077     0 days
6880     0 days
         ...   
6651   282 days
1809   285 days
6923   286 days
2494   293 days
5101   303 days
Name: time_to_dtg, Length: 6400, dtype: timedelta64[ns]

In [37]:
df_dtg.loc[[2494]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
2494,senate Bill S2505,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to name change upon marriage\n\n\nWome...,Anna M. Kaplan,"(D, IP, WF) 7th Senate District",62,0,True,2019-12-17,...,2019-02-27,returned to assembly,True,"Dec 20, 2019",False,no date,BILL NUMBER: S2505 REVISED 2/28/19\n\...,other spouse.\n\nSection 2 amends subdivision ...,S2505 (ACTIVE) - Bill Text\n \ndownlo...,293 days


In [38]:
df_dtg.loc[[1983]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
1983,senate Bill S2003,2013,https://www.nysenate.gov/legislation/bills/201...,Provides for a charge-back of refunds of taxes...,David Carlucci,(D) 0 Senate District,58,0,True,2014-12-17,...,2014-06-19,returned to senate,True,"Dec 29, 2014",False,no date,BILL NUMBER:S2003\n\nTITLE OF BILL: An act ...,refunds to such libraries. The inequity of req...,S2003 (ACTIVE) - Bill Text\n \ndownlo...,181 days
1983,senate Bill S2000D,2017,https://www.nysenate.gov/legislation/bills/201...,STATE OPERATIONS BUDGET,No sponsers found,No district found,57,0,True,2017-04-09,...,2017-04-09,substituted by a3000d,True,"Apr 20, 2017",False,no date,NaN,list index out of range,S2000 - Bill Text\n \ndownload pdf,0 days


In [39]:
df_dtg.loc[[6238]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
6238,senate Bill S6259D,2011,https://www.nysenate.gov/legislation/bills/201...,Enacts into law major components of legislatio...,No sponsers found,No district found,55,5,True,2012-03-30,...,2012-03-30,substituted by a9059d,True,"Mar 30, 2012",False,no date,BILL NUMBER:S6259\n\nTITLE OF BILL:\n\nAn act\...,"to amend the tax law, in relation\nto making t...",S6259 - Bill Text\n \ndownload pdf\n\...,0 days
6238,senate Bill S6249,2015,https://www.nysenate.gov/legislation/bills/201...,Establishes a tax credit for the purchase and ...,Robert G. Ortt,"(R, C, IP) 62nd Senate District",62,0,True,2016-11-16,...,2016-06-15,returned to assembly,False,no date,True,"Nov 28, 2016",BILL NUMBER: S6249\n\n TITLE OF BILL :\n\nAn ...,carbon used to heat buildings.\n\n PRIOR LEGIS...,S6249 (ACTIVE) - Bill Text\n \ndownlo...,154 days
6238,senate Bill S6249,2019,https://www.nysenate.gov/legislation/bills/201...,Clarifies the employee requirements for apprai...,James Sanders Jr.,(D) 10th Senate District,62,0,True,2019-12-10,...,2019-06-17,returned to assembly,True,"Dec 16, 2019",False,no date,BILL NUMBER: S6249\n\nSPONSOR: SANDERS\n \nTIT...,JUSTIFICATION:\n\nCurrent law has been interpr...,S6249 (ACTIVE) - Bill Text\n \ndownlo...,176 days


In [40]:
df_dtg[df_dtg.signed_by_gov == True].shape

(5458, 21)

In [41]:
df_dtg[df_dtg.signed_by_gov == True].head()

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
58,senate Bill S79A,2011,https://www.nysenate.gov/legislation/bills/201...,Authorizes the incorporation of benefit corpor...,Daniel L. Squadron,"(D, WF) 0 Senate District",62,0,True,2011-11-30,...,2011-06-17,returned to senate,True,"Dec 12, 2011",False,no date,BILL NUMBER:S79\n\nTITLE OF BILL:\nAn act\nto ...,Section 2 of the bill amends the New York Corp...,S79 - Bill Text\n \ndownload pdf\n\n\...,166 days
128,senate Bill S149B,2011,https://www.nysenate.gov/legislation/bills/201...,Relates to tax credits provided for solar ener...,George D. Maziarz,(R) 0 Senate District,60,0,True,2012-08-06,...,2012-06-20,returned to assembly,True,"Aug 17, 2012",False,no date,BILL NUMBER:S149\n\nTITLE OF BILL:\nAn act\nto...,feasible for the individual to lease the equip...,S149 - Bill Text\n \ndownload pdf\n\n...,47 days
154,senate Bill S175,2011,https://www.nysenate.gov/legislation/bills/201...,Prohibits state officers and employees from us...,George D. Maziarz,(R) 0 Senate District,62,0,True,2011-09-12,...,2011-06-21,returned to assembly,True,"Sep 23, 2011",False,no date,BILL NUMBER:S175\n\nTITLE OF BILL:\nAn act\nto...,"JUSTIFICATION:\nWhile during working hours, it...",S175 (ACTIVE) - Bill Text\n \ndownloa...,83 days
156,senate Bill S177A,2011,https://www.nysenate.gov/legislation/bills/201...,Reduces the required minimum age for obtaining...,George D. Maziarz,(R) 0 Senate District,53,7,True,2011-05-06,...,2011-05-03,returned to senate,True,"May 17, 2011",False,no date,BILL NUMBER:S177\n\nTITLE OF BILL: An act to ...,NaN,S177 - Bill Text\n \ndownload pdf\n\n...,3 days
184,senate Bill S205,2011,https://www.nysenate.gov/legislation/bills/201...,"Authorizes a county, city, town or village to ...",George D. Maziarz,(R) 0 Senate District,55,0,True,2011-04-15,...,2011-04-06,returned to assembly,True,"Apr 27, 2011",False,no date,BILL NUMBER:S205\n\nTITLE OF BILL:\nAn act\nto...,directed to be made and completed on or before...,S205 (ACTIVE) - Bill Text\n \ndownloa...,9 days


In [42]:
df_dtg[df_dtg.session == 2019].time_to_dtg.sort_values(ascending=False).head(20)

2494   293 days
1809   285 days
7104   282 days
3114   282 days
6651   282 days
7293   281 days
649    279 days
2366   269 days
3574   265 days
3237   263 days
3309   261 days
6696   261 days
6695   259 days
2486   246 days
4487   245 days
490    244 days
859    235 days
4172   235 days
2594   230 days
3129   230 days
Name: time_to_dtg, dtype: timedelta64[ns]

In [43]:
## https://www.nysenate.gov/legislation/bills/2019/s6662 ##

df_dtg.loc[[7293]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
7293,senate Bill S7314A,2011,https://www.nysenate.gov/legislation/bills/201...,Relates to self-funded student health benefit ...,James L. Seward,"(R, C, IP, RFM) 0 Senate District",60,0,True,2012-07-06,...,2012-06-20,returned to senate,True,"Jul 18, 2012",False,no date,BILL NUMBER:S7314\n\nTITLE OF BILL:\nAn act\nt...,section.\n\nSection 8004 requires colleges and...,S7314 - Bill Text\n \ndownload pdf\n\...,16 days
7293,senate Bill S7313,2013,https://www.nysenate.gov/legislation/bills/201...,Extends certain provisions authorizing the imp...,Jeffrey D. Klein,(D) 0 Senate District,58,0,True,2014-07-11,...,2014-06-10,returned to assembly,True,"Jul 22, 2014",False,no date,BILL NUMBER:S7313\n\nTITLE OF BILL: An act to...,FISCAL IMPLICATIONS: None.\n\nEFFECTIVE DATE: ...,S7313 (ACTIVE) - Bill Text\n \ndownlo...,31 days
7293,senate Bill S7310,2017,https://www.nysenate.gov/legislation/bills/201...,Relates to the frequency of inspections of art...,Jose Peralta,"(D, WF) 0 Senate District",61,0,True,2018-04-06,...,2018-02-12,returned to senate,True,"Apr 18, 2018",False,no date,BILL NUMBER: S7310\n\nSPONSOR: PERALTA\n \nTIT...,"EXISTING LAW:\n\nCurrently, General Business L...",S7310 (ACTIVE) - Bill Text\n \ndownlo...,53 days
7293,senate Bill S7304,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to membership on the public health and...,Brad Hoylman,"(D, WF) 27th Senate District",45,15,True,2020-12-03,...,2020-02-26,returned to assembly,True,"Dec 15, 2020",False,no date,BILL NUMBER: S7304\n\nSPONSOR: HOYLMAN\n \nTIT...,JUSTIFICATION:\n\nThe Public Health and Health...,S7304 (ACTIVE) - Bill Text\n \ndownlo...,281 days


In [44]:
## https://www.nysenate.gov/legislation/bills/2019/s6662 ##

df_dtg.loc[[6651]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
6651,senate Bill S6662,2019,https://www.nysenate.gov/legislation/bills/201...,Requires the department of health of the state...,Brian A. Benjamin,(D) 30th Senate District,62,0,True,2020-12-11,...,2020-03-04,returned to assembly,True,"Dec 23, 2020",False,no date,BILL NUMBER: S6662\n\nSPONSOR: BENJAMIN\n \nTI...,SUMMARY OF PROVISIONS:\n\nThe public health la...,S6662 (ACTIVE) - Bill Text\n \ndownlo...,282 days


In [45]:
## https://www.nysenate.gov/legislation/bills/2019/s3125 ##

df_dtg.loc[[3114]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
3114,senate Bill S3125,2019,https://www.nysenate.gov/legislation/bills/201...,Requires feminine hygiene products to be offer...,Alessandra Biaggi,"(D, WF) 34th Senate District",62,0,True,2019-12-06,...,2019-02-27,returned to assembly,False,no date,True,"Dec 13, 2019",BILL NUMBER: S3125\n\nSPONSOR: BIAGGI\n \nTITL...,"public schools, for students in grades 6-12. T...",S3125 (ACTIVE) - Bill Text\n \ndownlo...,282 days


In [46]:
## https://www.nysenate.gov/legislation/bills/2019/s7155

df_dtg.loc[[7104]]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
7104,senate Bill S7125,2011,https://www.nysenate.gov/legislation/bills/201...,"Relates to the definition of ""stallion"" for th...",Roy J. McDonald,"(R, C, IP) 0 Senate District",60,0,True,2012-07-20,...,2012-06-21,returned to assembly,True,"Aug 01, 2012",False,no date,BILL NUMBER:S7125\n\nTITLE OF BILL:\nAn act\nt...,the minimum lease period to one year is expect...,S7125 (ACTIVE) - Bill Text\n \ndownlo...,29 days
7104,senate Bill S7115,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to the dispensing of partially filled ...,Gustavo Rivera,"(D, WF) 33rd Senate District",61,0,True,2020-12-03,...,2020-02-25,returned to senate,False,no date,True,"Dec 15, 2020",BILL NUMBER: S7115\n\nSPONSOR: RIVERA\n \nTITL...,6816 of the Education Law by adding a referenc...,S7115 (ACTIVE) - Bill Text\n \ndownlo...,282 days


### Check to see if 'amendment' is in the URL

In [47]:
df_dtg[df_dtg.url.str.contains('amendment') == False]

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg
58,senate Bill S79A,2011,https://www.nysenate.gov/legislation/bills/201...,Authorizes the incorporation of benefit corpor...,Daniel L. Squadron,"(D, WF) 0 Senate District",62,0,True,2011-11-30,...,2011-06-17,returned to senate,True,"Dec 12, 2011",False,no date,BILL NUMBER:S79\n\nTITLE OF BILL:\nAn act\nto ...,Section 2 of the bill amends the New York Corp...,S79 - Bill Text\n \ndownload pdf\n\n\...,166 days
128,senate Bill S149B,2011,https://www.nysenate.gov/legislation/bills/201...,Relates to tax credits provided for solar ener...,George D. Maziarz,(R) 0 Senate District,60,0,True,2012-08-06,...,2012-06-20,returned to assembly,True,"Aug 17, 2012",False,no date,BILL NUMBER:S149\n\nTITLE OF BILL:\nAn act\nto...,feasible for the individual to lease the equip...,S149 - Bill Text\n \ndownload pdf\n\n...,47 days
154,senate Bill S175,2011,https://www.nysenate.gov/legislation/bills/201...,Prohibits state officers and employees from us...,George D. Maziarz,(R) 0 Senate District,62,0,True,2011-09-12,...,2011-06-21,returned to assembly,True,"Sep 23, 2011",False,no date,BILL NUMBER:S175\n\nTITLE OF BILL:\nAn act\nto...,"JUSTIFICATION:\nWhile during working hours, it...",S175 (ACTIVE) - Bill Text\n \ndownloa...,83 days
156,senate Bill S177A,2011,https://www.nysenate.gov/legislation/bills/201...,Reduces the required minimum age for obtaining...,George D. Maziarz,(R) 0 Senate District,53,7,True,2011-05-06,...,2011-05-03,returned to senate,True,"May 17, 2011",False,no date,BILL NUMBER:S177\n\nTITLE OF BILL: An act to ...,NaN,S177 - Bill Text\n \ndownload pdf\n\n...,3 days
184,senate Bill S205,2011,https://www.nysenate.gov/legislation/bills/201...,"Authorizes a county, city, town or village to ...",George D. Maziarz,(R) 0 Senate District,55,0,True,2011-04-15,...,2011-04-06,returned to assembly,True,"Apr 27, 2011",False,no date,BILL NUMBER:S205\n\nTITLE OF BILL:\nAn act\nto...,directed to be made and completed on or before...,S205 (ACTIVE) - Bill Text\n \ndownloa...,9 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,senate Bill S8831,2019,https://www.nysenate.gov/legislation/bills/201...,Requires judges who recuse themselves to provi...,Todd Kaminsky,(D) 9th Senate District,43,17,True,2020-12-11,...,2020-07-23,returned to senate,True,"Dec 23, 2020",False,no date,BILL NUMBER: S8831\n\nSPONSOR: KAMINSKY\n \nTI...,JUSTIFICATION:\n\nJudicial recusal is an impor...,S8831 (ACTIVE) - Bill Text\n \ndownlo...,141 days
8821,senate Bill S8832,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to the use of an electronic signature ...,John C. Liu,(D) 11th Senate District,57,3,True,2020-08-12,...,2020-07-23,returned to senate,True,"Aug 24, 2020",False,no date,BILL NUMBER: S8832\n\nSPONSOR: LIU\n \nTITLE O...,"However, tax preparers filing a NYS tax return...",S8832 (ACTIVE) - Bill Text\n \ndownlo...,20 days
8823,senate Bill S8834,2019,https://www.nysenate.gov/legislation/bills/201...,Authorizes former foster care youth to re-ente...,Velmanette Montgomery,(D) 0 Senate District,60,0,True,2020-12-03,...,2020-07-23,returned to senate,True,"Dec 15, 2020",False,no date,BILL NUMBER: S8834\n\nSPONSOR: MONTGOMERY\n \n...,social services department would be required t...,S8834 (ACTIVE) - Bill Text\n \ndownlo...,133 days
8824,senate Bill S8835,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to amending provisions regarding healt...,Luis R. Sepúlveda,(D) 32nd Senate District,45,15,True,2020-07-24,...,2020-07-23,returned to senate,True,"Aug 03, 2020",False,no date,BILL NUMBER: S8835\n\nSPONSOR: SEPULVEDA\n \nT...,relates to COVID-19; and removes the care of a...,S8835 (ACTIVE) - Bill Text\n \ndownlo...,1 days


# Veto proof bills that were vetoed
## First numbers by session years. Then vote analysis

Total number of vetoes

In [48]:
df_dtg[df_dtg.vetoed_by_gov == True].shape

(933, 21)

2011 Session vetos (2011 and 2012)

In [49]:
df_2011s_veto = df_dtg[df_dtg.session == 2011]
df_2011s_veto[df_2011s_veto.vetoed_by_gov == True].shape

(126, 21)

2013 Session vetos (2013 and 2014)

In [50]:
df_2013s_veto = df_dtg[df_dtg.session == 2013]
df_2013s_veto[df_2013s_veto.vetoed_by_gov == True].shape

(184, 21)

2015 Session vetos (2015 and 2016)

In [51]:
df_2015s_veto = df_dtg[df_dtg.session == 2015]
df_2015s_veto[df_2015s_veto.vetoed_by_gov == True].shape

(217, 21)

2017 Session vetos (2017 and 2018)

In [52]:
df_2017s_veto = df_dtg[df_dtg.session == 2017]
df_2017s_veto[df_2017s_veto.vetoed_by_gov == True].shape

(214, 21)

2019 Session vetoes (2019 and 2020)

In [53]:
df_2019s_veto = df_dtg[df_dtg.session == 2019]
df_2019s_veto[df_2019s_veto.vetoed_by_gov == True].shape

(192, 21)

### Vote Analysis

In [54]:
df_dtg.columns

Index(['bill_number', 'session', 'url', 'bill_title', 'spon_sen_name',
       'dist_name', 'aye_vote', 'nay_vote', 'dtg', 'dtg_date', 'lla_needed?',
       'lla_date', 'lla_action', 'signed_by_gov', 'signed_date',
       'vetoed_by_gov', 'vetoed_date', 'sponser_memo', 'sponser_memo2',
       'bill_text', 'time_to_dtg'],
      dtype='object')

In [55]:
df_dtg_vetoed = df_dtg[df_dtg.vetoed_by_gov == True]
df_dtg_vetoed.aye_vote.value_counts()

## For the output below, as a reminder, the VOTE TOTALS are on the left.
## The number of time they occur is on the right.

62    260
61    139
60    118
59     96
63     84
58     48
57     46
56     25
55     16
53     13
54     10
44      9
43      8
47      8
52      8
48      7
51      5
42      4
45      4
49      4
46      3
40      3
20      3
36      2
50      2
24      2
38      1
16      1
19      1
39      1
37      1
32      1
Name: aye_vote, dtype: int64

In [56]:
df_dtg_vetoed.shape

(933, 21)

In [57]:
## The senate can override a veto if it has more than 2/3 votes in favor of it. This next step
## deduce how many bills are potentially veto proof based on how the first floor vote went.

df_dtg_vetoed_potential_override = df_dtg_vetoed[df_dtg_vetoed.aye_vote.astype(int) > 41]
df_dtg_vetoed_potential_override.shape

(917, 21)

In [58]:
df_dtg_vetoed_potential_override.aye_vote.value_counts()

62    260
61    139
60    118
59     96
63     84
58     48
57     46
56     25
55     16
53     13
54     10
44      9
43      8
47      8
52      8
48      7
51      5
42      4
45      4
49      4
46      3
50      2
Name: aye_vote, dtype: int64

In [59]:
pd.set_option('display.max_colwidth', None)

In [60]:
df_dtg_vetoed_potential_override
for x in df_dtg_vetoed_potential_override:
    print(df_dtg_vetoed_potential_override.url.astype(str))

226      https://www.nysenate.gov/legislation/bills/2011/s247
275      https://www.nysenate.gov/legislation/bills/2011/s296
351      https://www.nysenate.gov/legislation/bills/2011/s372
431      https://www.nysenate.gov/legislation/bills/2011/s452
490      https://www.nysenate.gov/legislation/bills/2011/s511
                                ...                          
8108    https://www.nysenate.gov/legislation/bills/2019/s8119
8519    https://www.nysenate.gov/legislation/bills/2019/s8530
8641    https://www.nysenate.gov/legislation/bills/2019/s8652
8708    https://www.nysenate.gov/legislation/bills/2019/s8719
8713    https://www.nysenate.gov/legislation/bills/2019/s8724
Name: url, Length: 917, dtype: object
226      https://www.nysenate.gov/legislation/bills/2011/s247
275      https://www.nysenate.gov/legislation/bills/2011/s296
351      https://www.nysenate.gov/legislation/bills/2011/s372
431      https://www.nysenate.gov/legislation/bills/2011/s452
490      https://www.nysenate.go

In [69]:
pd.set_option('display.max_colwidth',50)

In [68]:
## 226 -- 139 yea/0 nay
## 275 -- 142 yes/2 nay
## 2019/S8119C -- 142 yes/2 nay -- COVID Bill
## 2019/S8652A -- 142 yes/0 nay

## DTG Month analysis

My conversation with Gerald Benjamin suggested that I should analyze during what months bills were sent to the governor. Perhaps then I can use those categories to see if there's any content classification. This section below spits out 12 CSV files for my dataframes

In [62]:
df_dtg.columns

Index(['bill_number', 'session', 'url', 'bill_title', 'spon_sen_name',
       'dist_name', 'aye_vote', 'nay_vote', 'dtg', 'dtg_date', 'lla_needed?',
       'lla_date', 'lla_action', 'signed_by_gov', 'signed_date',
       'vetoed_by_gov', 'vetoed_date', 'sponser_memo', 'sponser_memo2',
       'bill_text', 'time_to_dtg'],
      dtype='object')

In [64]:
df_dtg['month'] = df_dtg['dtg_date'].dt.month

In [65]:
df_dtg.month.value_counts()

7     1294
8      990
11     958
12     946
9      741
10     538
6      360
4      208
5      149
3      127
1       50
2       39
Name: month, dtype: int64

In [66]:
df_dtg_jan = df_dtg[df_dtg.month == 1]
df_dtg_feb = df_dtg[df_dtg.month == 2]
df_dtg_mar = df_dtg[df_dtg.month == 3]
df_dtg_apr = df_dtg[df_dtg.month == 4]
df_dtg_may = df_dtg[df_dtg.month == 5]
df_dtg_jun = df_dtg[df_dtg.month == 6]
df_dtg_jul = df_dtg[df_dtg.month == 7]
df_dtg_aug = df_dtg[df_dtg.month == 8]
df_dtg_sep = df_dtg[df_dtg.month == 9]
df_dtg_oct = df_dtg[df_dtg.month == 10]
df_dtg_nov = df_dtg[df_dtg.month == 11]
df_dtg_dec = df_dtg[df_dtg.month == 12]

In [67]:
df_dtg_jan.to_csv("all_legis_data_4.12.2021_jan_dtg.csv", sep="|")
df_dtg_feb.to_csv("all_legis_data_4.12.2021_feb_dtg.csv", sep="|")
df_dtg_mar.to_csv("all_legis_data_4.12.2021_mar_dtg.csv", sep="|")
df_dtg_apr.to_csv("all_legis_data_4.12.2021_apr_dtg.csv", sep="|")
df_dtg_may.to_csv("all_legis_data_4.12.2021_may_dtg.csv", sep="|")
df_dtg_jun.to_csv("all_legis_data_4.12.2021_jun_dtg.csv", sep="|")
df_dtg_jul.to_csv("all_legis_data_4.12.2021_jul_dtg.csv", sep="|")
df_dtg_aug.to_csv("all_legis_data_4.12.2021_aug_dtg.csv", sep="|")
df_dtg_sep.to_csv("all_legis_data_4.12.2021_sep_dtg.csv", sep="|")
df_dtg_oct.to_csv("all_legis_data_4.12.2021_oct_dtg.csv", sep="|")
df_dtg_nov.to_csv("all_legis_data_4.12.2021_nov_dtg.csv", sep="|")
df_dtg_dec.to_csv("all_legis_data_4.12.2021_dec_dtg.csv", sep="|")